# Running the final wake model

In [ ]:
from datetime import datetime

In [ ]:
# papermill parameters
# dir_name = "2024-12-16-21-26-57"
dir_name = "2024-12-11-21-31-37"
input_dir = "C:\\dev\\play\\brainwave-data"
force = False
print(f"Executing at {datetime.now()}")

In [ ]:
print(f"Executing at {datetime.now()}")

from models.util.papermill_util import exit_early
import os

physical_features_input_file = os.path.join(input_dir, dir_name, "raw.physical_features.csv")
post_yasa_file = os.path.join(input_dir, dir_name, "raw.post_yasa.csv")
post_human_file = os.path.join(input_dir, dir_name, "raw.post_human.csv")

print(f"Executing at {datetime.now()}")


if not (os.path.exists(physical_features_input_file)):
    print(f"Executing at {datetime.now()}")
    exit_early("Missing file")
if not os.path.exists(post_yasa_file):
    print(f"Executing at {datetime.now()}")
    exit_early("Missing file")


In [ ]:
# import pandas as pd
# import scrapbook as sb
#
output_file1 = os.path.join(input_dir, dir_name, "raw.final_wake_model.csv")
output_file2 = os.path.join(input_dir, dir_name, "raw.final_wake_model_post_human.csv")
#
# if os.path.exists(output_file1) and (os.path.exists(output_file2) or not os.path.exists(post_human_file)):
#     predictions_df = pd.read_csv(output_file1)
#     sb.glue("predictions_df", predictions_df)
#
#     if os.path.exists(output_file2):
#         predictions_post_human_df = pd.read_csv(output_file1)
#         sb.glue("predictions_post_human_df", predictions_post_human_df)
#
#     exit_early("Output file exists")

In [ ]:
%%prun -T "prof.cell7.txt"

print(f"Executing at {datetime.now()}")
# import run_post_yasa_pipeline
# import run_feature_pipeline
from importlib import reload
# import run_physical_features_pipeline
print(f"Executing at {datetime.now()}")

# reload(run_feature_pipeline)
# reload(run_physical_features_pipeline)
import pandas as pd
import os
# import sys
print(f"Executing at {datetime.now()}")

physical_features_input_file = os.path.join(input_dir, dir_name, "raw.physical_features.csv")
post_yasa_file = os.path.join(input_dir, dir_name, "raw.post_yasa.csv")
print(f"Executing at {datetime.now()}")


if not (os.path.exists(physical_features_input_file)):
    print(f"Executing at {datetime.now()}")
    exit_early("Missing file")
if not os.path.exists(post_yasa_file):
    print(f"Executing at {datetime.now()}")
    exit_early("Missing file")

from run_physical_features_pipeline import cached_physical_features_pipeline

#post_human_df = pd.read_csv(post_human_file)
input_file = os.path.join(input_dir, dir_name, "raw.fif")
post_yasa_df  = pd.read_csv(post_yasa_file)
physical_features_df, _ = cached_physical_features_pipeline(print, input_file, post_yasa_df, False, False, True)

#post_human_df.reset_index(drop=True, inplace=True)
physical_features_df.reset_index(drop=True, inplace=True)
df = pd.DataFrame.merge(post_yasa_df, physical_features_df, left_index=True, right_index=True)
# df = physical_features_df
#     #predictions_df = run_all(df)
#
#     return df, output_buffer.getvalue(), True
# else:
#     return None, output_buffer.getvalue(), "Missing file"
#
# # Testing
# predictions_df, stdout, success_or_msg = process_file_for_final_wake_model(input_dir, "2024-11-29-21-18-33", False)
# stdout, success_or_msg

In [ ]:
print(f"Executing at {datetime.now()}")

import models.util.features
reload(models.util.features)

from importlib import reload
import models.final_wake.final_wake_model
reload(models.final_wake.final_wake_model)
from models.final_wake.final_wake_model import create_and_add_all

models_and_data = create_and_add_all(df, True)


In [ ]:
print(f"Executing at {datetime.now()}")

from models.final_wake.final_wake_model import load_model
import xgboost as xgb

# all_models_filenames = [
#     # "models/PredictFinalWakeWithinNext10MinsAll_xgboost_model.cbm",
#     # "models/PredictFinalWakeWithinNext10MinsAll_xgboost_model.cbm",
#     # "models/PredictFinalWakeWithinNext10Mins_xgboost_model.cbm",
#     "models/PredictFinalWakeWithinNext10MinsEEGOnly_xgboost_model.cbm"
# ]
predictions_df = pd.DataFrame(index=df.index)

for md in models_and_data:
    model_filename = f"models/{md.name}_xgboost_model.cbm"
    model = load_model(model_filename)
    print(f"Loaded model {model_filename} which expects features {model.feature_names}")
    dmatrix = xgb.DMatrix(md.X)
    predictions = model.predict(dmatrix)
    predictions_df[md.name + "Prediction"] = predictions


In [ ]:
print(f"Executing at {datetime.now()}")

predictions_df.to_csv(os.path.join(input_dir, dir_name, output_file1))
import scrapbook as sb
sb.glue("predictions_df", predictions_df)

In [ ]:
print(f"Executing at {datetime.now()}")

import matplotlib.pyplot as plt

# Calculate the rolling average of the minsUntilWake column with a window size of 10
rolling_average = predictions_df['minsUntilWakePrediction'].rolling(window=10).mean()

# Plot the rolling average
plt.figure(figsize=(10, 6))
plt.plot(predictions_df['minsUntilWakePrediction'], label='minsUntilWakePrediction', alpha=0.5)
plt.plot(rolling_average, label='Rolling Average (window=10)', color='red')
plt.xlabel('Index')
plt.ylabel('minsUntilWakePrediction')
plt.title('Plot of minsUntilWake with Rolling Average')
plt.legend()
plt.show()

# Post-human tests (optional)

In [ ]:
import sys


if not os.path.exists(post_human_file):
    exit_early("Missing file")

In [ ]:
print(f"Executing at {datetime.now()}")

post_human_df = pd.read_csv(post_human_file)
merged = post_human_df.merge(predictions_df, left_index=True, right_index=True)
sb.glue("merged", merged)

In [ ]:
print(f"Executing at {datetime.now()}")

merged['minsUntilWakePredictionError'] = merged['minsUntilWakePrediction'] - merged['minsUntilWake']

In [ ]:
print(f"Executing at {datetime.now()}")

import matplotlib.pyplot as plt

# Calculate the rolling average of the minsUntilWake column with a window size of 10
rolling_average = merged['minsUntilWakePrediction'].rolling(window=10).mean()

# Plot the rolling average
plt.figure(figsize=(10, 6))
plt.plot(merged['minsUntilWake'], label='minsUntilWake real', alpha=0.5)
#plt.plot(merged['minsUntilWakePredictionError'], label='minsUntilWakePredictionError', alpha=0.5, color='green')
plt.plot(rolling_average, label='Rolling Average (window=10)', color='red')
plt.xlabel('Index')
plt.ylabel('minsUntilWake')
plt.title('Plot of minsUntilWake with Rolling Average')
plt.legend()
plt.show()

In [ ]:
print(f"Executing at {datetime.now()}")

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(merged['minsUntilWakePredictionError'], alpha=0.5, color='green')
plt.xlabel('Index')
plt.ylabel('minsUntilWakePredictionError')
plt.show()

In [ ]:
print(f"Executing at {datetime.now()}")

predictions_post_human_df = predictions_df.copy()
predictions_post_human_df['minsUntilWakeActual'] = merged['minsUntilWake']
predictions_post_human_df['minsUntilWakePredictionError'] = merged['minsUntilWakePredictionError']
predictions_post_human_df

In [ ]:
print(f"Executing at {datetime.now()}")

predictions_post_human_df.to_csv(output_file2)
# import scrapbook as sb
# sb.glue("predictions_post_human_df", predictions_post_human_df)

#

# JSON output

In [ ]:
predictions_post_human_df

In [ ]:
import json
# minsUntilWakeMeanPredictionError =
# minsUntilWakePredictionInitial=('minsUntilWakePrediction', lambda x: pd.to_timedelta(x.head(10).mean(), unit='m')),
# TempCount=('Temp', 'count'),
# HRCount=('HR', 'count'),

minsUntilWakePredictionInitial = predictions_post_human_df['minsUntilWakePrediction'].head(10).mean()

j = {
    'minsUntilWakeAllEpochsPredictionMeanError': float(predictions_post_human_df['minsUntilWakePredictionError'].abs().mean()),
    'minsUntilWakeInitialEpochsPredictionError': float(minsUntilWakePredictionInitial - predictions_post_human_df['minsUntilWakeActual'].head(10).mean()),
    'minsUntilWakeInitialEpochsPrediction': float(minsUntilWakePredictionInitial)
}

j

In [ ]:
filename = os.path.join(input_dir, dir_name, 'raw.final_wake_model_post_human.json')
print(f"Writing {filename}")
with open(filename, 'w') as file:
    json.dump(j, file, indent=4)  # `indent=4` makes the JSON file more readable